In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv001.ib.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/notebooks


In [2]:
import sys, os
import numpy as np

sys.path.append('..')
%aimport mre_pinn
%aimport train

Using backend: pytorch



In [12]:
%autoreload

data, test_data = mre_pinn.data.load_bioqic_dataset('../data/BIOQIC', 'fem_box', xyz_slice='2D')
data

Loading ../data/BIOQIC/four_target_phantom.mat
    __header__: <class 'bytes'>
    __version__: <class 'str'>
    __globals__: <class 'list'>
    u_ft: <class 'numpy.ndarray'> (100, 80, 10, 3, 6) complex128
Loading ../data/BIOQIC/fem_box_ground_truth.npy
     <class 'numpy.ndarray'> (6, 10, 80, 100) complex128
Multi frequency 2D
<xarray.Dataset>
Dimensions:    (frequency: 6, x: 80, y: 100, component: 2)
Coordinates:
  * frequency  (frequency) float64 50.0 60.0 70.0 80.0 90.0 100.0
  * x          (x) float64 0.0 0.001 0.002 0.003 ... 0.076 0.077 0.078 0.079
  * y          (y) float64 0.0 0.001 0.002 0.003 ... 0.096 0.097 0.098 0.099
    z          float64 0.0
  * component  (component) <U1 'z' 'y'
Data variables:
    u          (frequency, x, y, component) complex128 (7.435908284487881e-15...
    mu         (frequency, x, y) complex128 (3000+314.1592653589793j) ... (30...


<xarray.Dataset>
Dimensions:    (frequency: 6, x: 80, y: 100, component: 2)
Coordinates:
  * frequency  (frequency) float64 50.0 60.0 70.0 80.0 90.0 100.0
  * x          (x) float64 0.0 0.001 0.002 0.003 ... 0.076 0.077 0.078 0.079
  * y          (y) float64 0.0 0.001 0.002 0.003 ... 0.096 0.097 0.098 0.099
    z          float64 0.0
  * component  (component) <U1 'z' 'y'
Data variables:
    u          (frequency, x, y, component) complex128 (7.435908284487881e-15...
    mu         (frequency, x, y) complex128 (3000+314.1592653589793j) ... (30...
    Lu         (frequency, x, y, component) complex128 (-0.001673382030335908...
    Mu         (frequency, x, y) complex128 (2832.4756903541183-2515.88388301...